# Example of Lagrangians for Optimization
Peter Sharpe

## Problem
Here, we study the following problem, taken from MIT 16.888, Assignment 2 (Spring 2020):

Consider the simple constrained optimization problem of minimizing the surface area of a cylinder subject to an equality constraint on its volume:

$\text{min}_x f(x_1, x_2) = 2 \pi x_1 (x_1 + x_2)$

$\text{subject to: } h(x_1, x_2) = \pi x_1^2x_2 = 0$

where $x_1$ is the radius of the cylinder, $x_2$ is the height of the cylinder, and V is the required volume. Formulate the Lagrangian function, derive the optimality conditions, and solve the resulting system of equations to determine the dimensions of the minimum-surface-area cylinder that has a volume of 1 liter (1000 cm^3).

## Solution via KKT Condition Derivation

First, we import and set up libraries:

In [21]:
import numpy as np
import sympy as sp
sp.init_printing()
pi = sp.pi

We first formulate the Lagrangian of this problem and derive the optimality conditions:

In [22]:
x1 = sp.Symbol("x_1", real=True)
x2 = sp.Symbol("x_2", real=True)
l1 = sp.Symbol("lambda_1", real=True)
V = 0.001 # m^3, or 1 liter

We find $f(x_1, x_2)$:

In [23]:
f = 2 * pi * x1 * (x1 + x2)
f

and $h(x_1, x_2)$:

In [24]:
h = pi * x1 ** 2 * x2 - V
h

Now, we find the Lagrangian as a function of some Lagrange multiplier $\lambda_1$:



In [25]:
L = f + l1 * h
L

Recalling the definition of a Lagrangian, we need to find the stationary point $\vec{x}^*$ of $L(\vec{x}, \lambda_1)$ to solve our original constrained problem.

Since we have no inequality constraints, we can use a simplified version of the KKT conditions (the Lagrange conditions) as our optimality conditions:

We have an optimum $\vec{x}^*$ if:
1. $\vec{x}^*$ is feasible, meaning $h(\vec{x}^*) = 0$.
2. $\nabla L(\vec{x}, \lambda_1) = 0$

Now, we have three equations and three variables, so we can solve this nonlinear system:

In [26]:
eqns = [
    h,
    sp.diff(L, x1),
    sp.diff(L, x2)
]
eqns

In [27]:
# Exact symbolic
from sympy.solvers import solve
sol = solve(eqns, [x1, x2, l1])

# # Numerical
# from sympy.solvers import nsolve
# nsolve(eqns, [x1, x2, l1], [0.01, 0.01, 1])

x1_sol, x2_sol, l1_sol = sol[0]
print("x1_sol =", x1_sol)
print("x2_sol =", x2_sol)
print("l1_sol =", l1_sol)

x1_sol = 0.0541926070139289
x2_sol = 0.108385214027858
l1_sol = -36.9054029728806


So the 1L cylinder container with minimum surface area has a radius of 5.42 cm and a height of 10.84 cm. And the sensitivity of surface area to volume is -36.91 $m^2 / m^3$. That seems about right!

## Solution as a Geometric Program

We formulate the cylinder optimization problem as a geometric program now. We need to rewrite the program just a bit such that the $h(x_1, x_2)$ formulation is GP-compatible:

In [28]:
import cvxpy as cp

x1 = cp.Variable(pos=True)
x2 = cp.Variable(pos=True)
V = 0.355 * 0.001 # m^3

f = 2 * np.pi * x1 * (x1 + x2)
h = np.pi * x1 ** 2 * x2

constraints = [h == V]
obj = cp.Minimize(f)

prob = cp.Problem(obj, constraints)
prob.solve(gp=True)
print("x1_sol =", x1.value)
print("x2_sol =", x2.value)

x1_sol = 0.03837135935431579
x2_sol = 0.07674748102377016


We find that we get the same solution as before. 

## Solution as a General NLP

We can also efficiently solve the problem with a general nonlinear method:

In [29]:
import casadi as cas
opti = cas.Opti()

x1 = opti.variable()
x2 = opti.variable()
opti.set_initial(x1, 1)
opti.set_initial(x2, 1)

V = 0.355 * 0.001

opti.minimize(2*cas.pi*x1*(x1+x2))

opti.subject_to(cas.pi*x1**2*x2-V==0)

opti.solver('ipopt')
sol = opti.solve()
print("x1_sol =", sol.value(x1))
print("x2_sol =", sol.value(x2))

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1